In [ ]:
# pip install pandas numpy datetime dash plotly

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import plotly.graph_objects as go
import plotly.express as px

from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# --------------------------------------------------
# Load & preprocess data
# --------------------------------------------------
df = pd.read_csv("global_power_plant_database.csv")

# Filter to USA
usa = df[df["country"] == "USA"].copy()

# Ensure numeric year and drop rows without essentials
usa["commissioning_year"] = pd.to_numeric(usa["commissioning_year"], errors="coerce")
usa = usa.dropna(subset=["latitude", "longitude", "capacity_mw", "commissioning_year"])

# Age for hover (optional)
current_year = datetime.now().year
usa["age_years"] = current_year - usa["commissioning_year"]

# Fuel list
fuels = sorted(usa["primary_fuel"].dropna().unique())

# Color map per fuel (consistent)
palette = (
    px.colors.qualitative.Set2
    + px.colors.qualitative.Set1
    + px.colors.qualitative.Dark24
)
color_map = {fuel: palette[i % len(palette)] for i, fuel in enumerate(fuels)}

# Year range for slider
min_year = int(usa["commissioning_year"].min())
max_year = int(usa["commissioning_year"].max())

# Capacity range for MW filter
min_cap = float(usa["capacity_mw"].min())
max_cap = float(usa["capacity_mw"].max())

# --------------------------------------------------
# Dash app
# --------------------------------------------------
app = Dash(__name__)

app.layout = html.Div(
    style={"font-family": "Arial, sans-serif", "padding": "10px"},
    children=[
        html.H2("US Power Plants – Interactive Map"),

        html.Div(
            style={"display": "flex", "gap": "30px", "flex-wrap": "wrap"},
            children=[
                # Fuel multi-select dropdown
                html.Div(
                    style={"minWidth": "250px"},
                    children=[
                        html.Label("Fuel types to display:"),
                        dcc.Dropdown(
                            id="fuel-dropdown",
                            options=[{"label": f, "value": f} for f in fuels],
                            value=fuels,         # default: all fuels selected
                            multi=True,
                            placeholder="Select one or more fuels",
                        ),
                    ],
                ),

                # Year range slider (twice as long, sparser marks)
html.Div(
    style={"minWidth": "350px", "width": "700px"},  # make the container wide
    children=[
        html.Label("Commissioning year range:"),
        dcc.RangeSlider(
            id="year-range",
            min=min_year,
            max=max_year,
            step=1,
            value=[min_year, max_year],
            marks={
                y: str(y)
                for y in range(
                    (min_year // 10) * 10,
                    max_year + 1,
                    10      # marks every 10 years -> fewer labels
                )
            },
            allowCross=False,
            tooltip={"placement": "bottom", "always_visible": False},
            # NOTE: no style= here, style lives on the wrapping Div
        ),
    ],
),

                # Minimum MW filter
                html.Div(
                    style={"minWidth": "250px"},
                    children=[
                        html.Label("Minimum capacity (MW):"),
                        dcc.Slider(
                            id="min-mw",
                            min=0,
                            max=max_cap,
                            step=50,
                            value=0,
                            tooltip={"placement": "bottom", "always_visible": False},
                        ),
                        html.Div(
                            id="min-mw-label",
                            style={"marginTop": "5px", "fontSize": "0.9em"},
                        ),
                    ],
                ),
            ],
        ),

        html.Hr(),

        dcc.Graph(
            id="plant-map",
            style={"width": "1200px", "height": "720px"},
        ),
    ],
)

# --------------------------------------------------
# Callback to update MW label
# --------------------------------------------------
@app.callback(
    Output("min-mw-label", "children"),
    Input("min-mw", "value"),
)
def update_min_mw_label(min_mw):
    return f"Showing plants with capacity ≥ {min_mw:.0f} MW"


# --------------------------------------------------
# Callback to update the map
# --------------------------------------------------
@app.callback(
    Output("plant-map", "figure"),
    [
        Input("fuel-dropdown", "value"),
        Input("year-range", "value"),
        Input("min-mw", "value"),
    ],
)
def update_map(selected_fuels, year_range, min_mw):
    # Handle empty selection (show nothing)
    if not selected_fuels:
        filtered = usa.iloc[0:0].copy()
    else:
        start_year, end_year = year_range
        filtered = usa[
            (usa["primary_fuel"].isin(selected_fuels))
            & (usa["commissioning_year"] >= start_year)
            & (usa["commissioning_year"] <= end_year)
            & (usa["capacity_mw"] >= min_mw)
        ].copy()

    # If no plants after filtering, create an empty figure with same layout
    if filtered.empty:
        fig = go.Figure()
        fig.update_layout(
            width=1200,
            height=720,
            title="No plants match the current filters",
            geo=dict(
                scope="usa",
                projection_type="albers usa",
                landcolor="rgb(240,240,240)",
                lakecolor="rgb(220,220,220)",
                bgcolor="rgba(0,0,0,0)",
            ),
        )
        return fig

    # --- NO top-20 labels anymore ---
    # We just plot markers; no text labels.
    # Bubble size scaling (max ~20) within filtered subset
    max_cap_filtered = float(filtered["capacity_mw"].max())

    def size_scale(cap_series):
        return 5 + 15 * np.sqrt(cap_series / max_cap_filtered)

    fig = go.Figure()

    for fuel in sorted(filtered["primary_fuel"].dropna().unique()):
        sub = filtered[filtered["primary_fuel"] == fuel].copy()
        sizes = size_scale(sub["capacity_mw"])

        fig.add_trace(
            go.Scattergeo(
                lon=sub["longitude"],
                lat=sub["latitude"],
                hovertext=(
                    "Name: " + sub["name"]
                    + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                    + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                    + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                    + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
                ),
                mode="markers",     # <-- markers only (no text labels)
                marker=dict(
                    size=sizes,
                    color=color_map.get(fuel, "gray"),
                    opacity=0.75,
                    line=dict(width=0.2, color="black"),
                ),
                name=fuel,
                showlegend=True,
            )
        )

    fig.update_layout(
        width=1200,
        height=720,
        title="US Power Plants – Filtered by Fuel, Year, and Capacity",
        geo=dict(
            scope="usa",
            projection_type="albers usa",
            landcolor="rgb(240,240,240)",
            lakecolor="rgb(220,220,220)",
            bgcolor="rgba(0,0,0,0)",
        ),
        legend=dict(
            title="Primary Fuel",
            orientation="v",
            x=0.99,
            xanchor="right",
            y=0.99,
            yanchor="top",
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="black",
            borderwidth=1,
        ),
        margin=dict(l=50, r=200, t=80, b=60),
    )

    return fig


if __name__ == "__main__":
    app.run(debug=True)
